# POC - EPFO Question and Answer System
## Account (UAN) (creation, documents required, claims), KYC (procedure, update)
This is an end to end LLM project based on Google Palm and Langchain. In this project a question and answer system related to EPFO (Employee's Provident Fund Organization) is developed. EPFO is one of the World's largest Social Security Organisations in terms of clientele and the volume of financial transactions undertaken. In the developed project questions related to account (UAN) (creation, documents required, claims), KYC (procedure, update) etc. are tried to answered using google palm large language model.

## Project Architecture:
1. **CSV loading :** CSV loader from langchain document loader will load the csv question and answer file.
2. **Database questions embedding :** Questions from CSV question and answer file will be embedded using <u>huggingface embeding</u>.
3. **Vector Database :** Embedded questions and corresponding answers will be stored using <u>FAISS</u>.
4. **Creating a retrieval chain :** Using a <u>prompt template</u>  and <u>google palm api</u> retrieval chain will be prepared.

## Output:
Output will be an answer based on the input question. Following will happen in the background.
1. A question asked to the retrieval chain will try to find the similar questions from the vector database.
2. Corresponding answers from the vector database of the relevant questions from step 1 will be outputted nicely using google palm llm.

## Setting up the API keys

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH_PROJECT")

In [3]:
# Check
from langchain.chat_models import init_chat_model
from rich.markdown import Markdown
llm = init_chat_model("openai:gpt-4.1-nano", temperature=0)
result = llm.invoke("What is an agent?")
Markdown(result.content)

An agent is an entity that perceives its environment through sensors and acts upon it through actuators to achieve 
specific goals or perform tasks. In various contexts, the term "agent" can have different meanings:                

 1 In Artificial Intelligence and Computer Science:                                                                
   An agent is a software program or system that autonomously makes decisions and performs actions based on its    
   perceptions and predefined objectives. Examples include virtual assistants, autonomous robots, and intelligent  
   agents in multi-agent systems.                                                                                  
 2 In Economics and Business:                                                                                      
   An agent is an individual or organization that acts on behalf of another (the principal), such as a salesperson 
   representing a company or a broker acting for clients.                                                          
 3 In General Usage:                                                                                               
   An agent can refer to any person or thing that causes an effect or performs an action.                          

Overall, the core idea is that an agent interacts with its environment to accomplish specific tasks or goals.

## Creating a vector database and question embedding

### First time creating the embeddings and storing on local

In [3]:
from langchain_community.document_loaders import CSVLoader
# Load the data from EPFO faq's
# loader = CSVLoader(file_path='C:\Swapnil\GenerativeAI\Practice_1\EPFO_Chatbot_Project\EPFO_FAQs.csv', encoding='unicode_escape', source_column="Question ")
loader = CSVLoader(file_path='EPFO_FAQs.csv', encoding='unicode_escape', source_column="Question ")

# Store the loaded data in the 'data' variable
data = loader.load()

# correcting the rows as there are only specific number of questions
data=data[:41]

# lets check the read data
data

[Document(metadata={'source': 'What is Universal Account Number or UAN', 'row': 0}, page_content='Question: What is Universal Account Number or UAN\nAnswer: UAN is 12-digit number provided to each member of EPFO. The UAN acts as an umbrella for the multiple Member IDs allotted to an individual. This number acts as a pivot to link multiple Member Identification Numbers (Member Id) allotted to a single member under single Universal Account Number. UAN duly seeded with KYC detail. This enables the member to avail various online services directly without the need for any intermediation by the employer.'),
 Document(metadata={'source': 'What is KYC', 'row': 1}, page_content='Question: What is KYC\nAnswer: Know Your Customer or KYC is a one-time process which helps in identity verification of subscribers by linking UAN with KYC details. The Employees / Employers need to provide KYC details viz., Aadhaar, PAN, Bank etc., for unique identification of the employees enabling seamless online serv

In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_community.vectorstores import FAISS
# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,embedding=embeddings)

# Specify the relative path (current folder)
db_path = "faiss_db_epfo"

# Save the FAISS vector store locally
vectordb.save_local(db_path)
print("Saving FAISS DB to:", os.path.abspath(db_path))

Saving FAISS DB to: c:\Swapnil\GenerativeAI\AnsweringQuestionsUsingLLM\faiss_db_epfo


### Using the created database without need of embedding

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
db_path = "faiss_db_epfo"
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

print("FAISS DB loaded from:", os.path.abspath(db_path))

FAISS DB loaded from: c:\Swapnil\GenerativeAI\AnsweringQuestionsUsingLLM\faiss_db_epfo


### Defining the retriever and checking the retrived documents

In [5]:
retriever = vectordb.as_retriever()

In [6]:
# Sample question and corresponding searched questions in the vector database
rdocs = retriever.get_relevant_documents("What should I do if I change my job")
rdocs

C:\Users\HP\AppData\Local\Temp\ipykernel_7404\2540997025.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rdocs = retriever.get_relevant_documents("What should I do if I change my job")


[Document(id='80e2fcec-8993-45a1-af15-7d8de227e986', metadata={'source': 'What is to be done in case I change the job and join somewhere else', 'row': 33}, page_content='Question: What is to be done in case I change the job and join somewhere else\nAnswer: You need to simply declare your UAN with your subsequent employer.'),
 Document(id='f4cf3d05-6c92-4ff7-9220-08b5534806eb', metadata={'source': 'I have changed my job. Should I activate my UAN again', 'row': 16}, page_content='Question: I have changed my job. Should I activate my UAN again\nAnswer: UAN has to be activated only once. You do not have to re-activate it every time you switch jobs.'),
 Document(id='0e39744d-1c28-4851-8177-d8ec6ce59e80', metadata={'source': 'My employer has not updated my date of exit despite lapse of two months after leaving job. How can I update my date of exit', 'row': 27}, page_content='Question: My employer has not updated my date of exit despite lapse of two months after leaving job. How can I update 

In [7]:
rdocs = retriever.get_relevant_documents("What is the procedure to change the password and can i link two mobile phones to a single account")
rdocs

[Document(id='51c16bc4-fedd-405c-8b38-40b1cec55fab', metadata={'source': 'What to do if I forgot my password and my registered mobile with UAN has also changed', 'row': 32}, page_content='Question: What to do if I forgot my password and my registered mobile with UAN has also changed\nAnswer: Please click on \x93Forgot Password\x94 at Member Interface of Unified Portal. Provide your UAN with CAPTCHA. System will ask whether OTP is to be sent on registered mobile or some other mobile. System will ask to enter your basic details (Name, DOB and Gender). After successful matching of basic details system will ask to provide your Aadhar or PAN. If KYC details are matched system will ask new mobile number and OTP will be sent to the new mobile. After successful verification of OTP, you can reset your password.'),
 Document(id='4b8a7d7c-aa36-48bf-9aa0-a8fe9e8d5a55', metadata={'source': 'How to change my UAN linked mobile number', 'row': 28}, page_content='Question: How to change my UAN linked m

In [8]:
rdocs = retriever.get_relevant_documents("How to link an AADHAR with UAN")
rdocs

[Document(id='89d20529-99f5-4ba5-afe3-2c2160ed4f0e', metadata={'source': 'How can I seed my KYC details with UAN', 'row': 12}, page_content='Question: How can I seed my KYC details with UAN\nAnswer: o Login to your EPF account at the unified member portal o Click on the \x93KYC\x94 option in the \x93Manage\x94 section o You can select the details (PAN, Bank Account, Aadhar etc) which you want to link with UAN o Fill in the requisite fields o Now click on the \x93Save\x94 option o Your request will be displayed in \x93KYC Pending for Approval\x94 o Once employer approves the details the message will be changed to \x93Digitally approved by the employer\x94 o Once UIDAI confirms your details, \x93Verified by UIDAI\x94 is displayed against your Aadhaar.'),
 Document(id='a6e72e55-791c-44e1-a90a-b7c8e93e9098', metadata={'source': 'What can I do if my UAN is not seeded with Aadhaar', 'row': 10}, page_content='Question: What can I do if my UAN is not seeded with Aadhaar\nAnswer: Member can him

## Create RetrievalQA chain along with prompt template

In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import init_chat_model

In [10]:
llm = init_chat_model("openai:gpt-4.1-nano", temperature=0.3,max_retries=2)

In [11]:
prompt_template = """Given the following context and a question, generate an answer based on this context only. Context is 
relevent FAQ's found from employee provident fund FAQ's database. Read the context in details and come up with the answer from 
the "Answer" section in the context source document. You can elaborate your answers in maximum 2 sentences. If the relevent context
is not found to the asked question, kindly state "I do not know.".

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [12]:
chain("What should I do if I change my job")

C:\Users\HP\AppData\Local\Temp\ipykernel_7404\1425106059.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("What should I do if I change my job")


{'query': 'What should I do if I change my job',
 'result': 'You need to simply declare your UAN with your subsequent employer.',
 'source_documents': [Document(id='80e2fcec-8993-45a1-af15-7d8de227e986', metadata={'source': 'What is to be done in case I change the job and join somewhere else', 'row': 33}, page_content='Question: What is to be done in case I change the job and join somewhere else\nAnswer: You need to simply declare your UAN with your subsequent employer.'),
  Document(id='f4cf3d05-6c92-4ff7-9220-08b5534806eb', metadata={'source': 'I have changed my job. Should I activate my UAN again', 'row': 16}, page_content='Question: I have changed my job. Should I activate my UAN again\nAnswer: UAN has to be activated only once. You do not have to re-activate it every time you switch jobs.'),
  Document(id='0e39744d-1c28-4851-8177-d8ec6ce59e80', metadata={'source': 'My employer has not updated my date of exit despite lapse of two months after leaving job. How can I update my date o

In [13]:
chain("What is the procedure to change the password")

{'query': 'What is the procedure to change the password',
 'result': 'I do not know.',
 'source_documents': [Document(id='51c16bc4-fedd-405c-8b38-40b1cec55fab', metadata={'source': 'What to do if I forgot my password and my registered mobile with UAN has also changed', 'row': 32}, page_content='Question: What to do if I forgot my password and my registered mobile with UAN has also changed\nAnswer: Please click on \x93Forgot Password\x94 at Member Interface of Unified Portal. Provide your UAN with CAPTCHA. System will ask whether OTP is to be sent on registered mobile or some other mobile. System will ask to enter your basic details (Name, DOB and Gender). After successful matching of basic details system will ask to provide your Aadhar or PAN. If KYC details are matched system will ask new mobile number and OTP will be sent to the new mobile. After successful verification of OTP, you can reset your password.'),
  Document(id='8391c44d-e85e-42cf-b6d7-635356c8f406', metadata={'source': '

In [14]:
chain("How to link an AADHAR with UAN")

{'query': 'How to link an AADHAR with UAN',
 'result': 'You can seed your UAN with Aadhaar by logging into the EPF member portal, selecting the "KYC" option under the "Manage" section, filling in your Aadhaar details, and saving the information. Once your employer approves and UIDAI verifies your Aadhaar, the linkage will be complete.',
 'source_documents': [Document(id='89d20529-99f5-4ba5-afe3-2c2160ed4f0e', metadata={'source': 'How can I seed my KYC details with UAN', 'row': 12}, page_content='Question: How can I seed my KYC details with UAN\nAnswer: o Login to your EPF account at the unified member portal o Click on the \x93KYC\x94 option in the \x93Manage\x94 section o You can select the details (PAN, Bank Account, Aadhar etc) which you want to link with UAN o Fill in the requisite fields o Now click on the \x93Save\x94 option o Your request will be displayed in \x93KYC Pending for Approval\x94 o Once employer approves the details the message will be changed to \x93Digitally approv

In [15]:
chain("What if I purchase a mobile phone, do I need to create an account")

{'query': 'What if I purchase a mobile phone, do I need to create an account',
 'result': 'I do not know.',
 'source_documents': [Document(id='faa0ca22-99aa-4645-8b5b-b3b3aa0ba308', metadata={'source': 'Can I apply online claim if my mobile is not linked with Aadhaar', 'row': 29}, page_content='Question: Can I apply online claim if my mobile is not linked with Aadhaar\nAnswer: No, you cannot submit online claim if your mobile is not linked with Aadhaar. At the time of claim submission, OTP is sent to Aadhaar linked mobile only.'),
  Document(id='078a220f-ad63-40e5-9d1f-cb3e63d16430', metadata={'source': 'What to do if my Aadhaar is not linked with mobile', 'row': 30}, page_content='Question: What to do if my Aadhaar is not linked with mobile\nAnswer: Please visit your nearest Aadhaar Service Centre. For more details you can visit official website of UIDAI https://uidai.gov.in.'),
  Document(id='51c16bc4-fedd-405c-8b38-40b1cec55fab', metadata={'source': 'What to do if I forgot my passwo

In [16]:
chatbot_result=chain("How to link an AADHAR with UAN")

In [17]:
chatbot_result

{'query': 'How to link an AADHAR with UAN',
 'result': 'You can link your Aadhaar with UAN by logging into your EPF account at the unified member portal, selecting the "KYC" option under the "Manage" section, filling in your Aadhaar details, and then saving the information. Once your employer approves the details and UIDAI confirms your Aadhaar, the linkage will be completed.',
 'source_documents': [Document(id='89d20529-99f5-4ba5-afe3-2c2160ed4f0e', metadata={'source': 'How can I seed my KYC details with UAN', 'row': 12}, page_content='Question: How can I seed my KYC details with UAN\nAnswer: o Login to your EPF account at the unified member portal o Click on the \x93KYC\x94 option in the \x93Manage\x94 section o You can select the details (PAN, Bank Account, Aadhar etc) which you want to link with UAN o Fill in the requisite fields o Now click on the \x93Save\x94 option o Your request will be displayed in \x93KYC Pending for Approval\x94 o Once employer approves the details the messa

In [18]:
chatbot_result['result']

'You can link your Aadhaar with UAN by logging into your EPF account at the unified member portal, selecting the "KYC" option under the "Manage" section, filling in your Aadhaar details, and then saving the information. Once your employer approves the details and UIDAI confirms your Aadhaar, the linkage will be completed.'